In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master/

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 43.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━

In [4]:
!pip install rank_bm25 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [6]:
import pandas as pd
import numpy as np

def calculate_accuracy(df_json, k=100):
  true_cnt = 0
  true_k_cnt = 0
  for i in range(len(df_json)): # iterate per row
    flag=0  
    for j in range(len(df_json[i]['ctxs'])): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        flag=1
        true_cnt += 1
        break
        
    if flag==1:
      for m in range(k):
        if df_json[i]['ctxs'][m]['has_answer']:
          true_k_cnt += 1
          break
  return true_k_cnt / true_cnt

def calculate_precision(df_json, k=100):
  precision = 0
  for i in range(len(df_json)): # iterate per row
    true_cnt = 0
    pos_lst = [1 for x in df_json[i]['ctxs'] if x['has_answer']]
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        true_cnt += 1
    if len(pos_lst):
      curr_precision = true_cnt / len(pos_lst)
    else:
      curr_precision = 0
    precision += curr_precision
  return precision / len(df_json)

def calculate_mrr(df_json, k=100):
  mrr = 0
  for i in range(len(df_json)): # iterate per row
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        mrr += 1 / (j + 1) # (j+1)-th rank
        break
  return mrr / len(df_json)

def calculate_map(df_json, k=100):
  map = 0
  for i in range(len(df_json)): # iterate per row
    true_cnt = 0
    curr_precision = 0
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        true_cnt += 1
        curr_precision += true_cnt / (j + 1)
    if true_cnt:
      map += curr_precision / true_cnt
  return map / len(df_json)

def calculate_ndcg(df_json, k=100):
  ndcg = 0
  for i in range(len(df_json)): # iterate per row
    dcg = 0
    true_cnt = 0
    ideal_dcg = 1
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        dcg += 1 / np.log2(j + 2)
        ideal_dcg += 1 / np.log2(true_cnt + 2)
        true_cnt += 1
    if ideal_dcg or dcg:
      ndcg += dcg / ideal_dcg
  return ndcg / len(df_json)


In [13]:
import json
from rank_bm25 import BM25Okapi

def tokenizer(sent):
  return sent.split(" ")

def reading_json_file(path):
    queryy=[]
    answerr=[]
    
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        print(len(data))
        corpus = [[['a' for _ in range(4)] for _ in range(100)] for _ in range(len(data))]
        bm25_corpus = [['a' for _ in range(100)] for _ in range(len(data))] 
        for i in range(len(data)):
            #queryy.append(data[i]["question"])
            #queryy.append(data[i]["expanded_query"])
            #queryy.append(data[i]["expanded_query_2"])
            #queryy.append(data[i]["expanded_query_gpt"])
            queryy.append(data[i]["expanded_query_3"])
            #queryy.append(data[i]["expanded_query_4"])
            #answerr.append(data[i]["answers"]) #json 파일 answerr 
            for j in range(100):
                corpus[i][j][0]=data[i]["ctxs"][j]["id"]
                corpus[i][j][1]=data[i]["ctxs"][j]["title"]
                corpus[i][j][2]=data[i]["ctxs"][j]["text"]
                corpus[i][j][3]=data[i]["ctxs"][j]["has_answer"]
                bm25_corpus[i][j]=corpus[i][j][2]
    # tokenized_corpus = [tokenizer(doc) for doc in bm25_corpus]
    # bm25 = BM25Okapi(tokenized_corpus)
    
    # for i in range(len(queryy)): 
    #   tokenized_query = tokenizer(queryy[i])
    #   doc_scores = bm25.get_scores(tokenized_query)
    #   for j in range(100): 
    #     bm25_scores.append(doc_scores[j])

    # for i in range(len(corpus)):
    #   corpus[i] = sorted(corpus[i], key=lambda x: x[3], reverse=True)
      
    # #query당 100개의 text의 bm25 score를 계산 -> 이를 바탕으로 sorting -> somewhat reranking 
    return queryy, answerr, corpus, data, bm25_corpus

In [14]:
file_path='/content/drive/MyDrive/Colab Notebooks/NQ_test_benchmark_withgpt_deduplicated_final.json'
queryy, answerr, corpus, data, bm25_corpus = reading_json_file(file_path)
print(queryy[0])

272
The southwest monsoon blow over Nigeria during summer
the south west wind blows across nigeria between


In [9]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


In [15]:
from rank_bm25 import BM25Okapi
import numpy as py 
import json 

def tokenizer(sent):
    return sent.split(" ") 
    #sent.split("\n"): 문장 전체를 받아줌 
    #sent.split(""): 단어 별로 받음 (문장 -> 단어로 쪼개짐)
  
bm25_scores = [] 
result = [] 

for q in range(len(queryy)): 
  #print(len(bm25_corpus[q]))
  tokenized_corpus = [tokenizer(doc) for doc in bm25_corpus[q]]
  #print(tokenized_corpus)
  bm25 = BM25Okapi(tokenized_corpus)
    
  query = queryy[q]
  #answer = answerr[q]
  # print(answer)
  tokenized_query = query.split(" ")
  # print("######################################")
  # print(tokenized_query)
    # print("######################################")
  doc_scores = bm25.get_scores(tokenized_query)
  
  s = doc_scores.argsort() ##오름차순으로 정렬된 index 
  sorted_scores = doc_scores[s][::-1] #내림차순 
  #print(sorted_scores)

  # print(sorted_scores)
  
  sorted_id = [corpus[q][i][0] for i in s[::-1]]
  sorted_title = [corpus[q][i][1] for i in s[::-1]]
  sorted_text = [corpus[q][i][2] for i in s[::-1]]
  sorted_has_answer = [corpus[q][i][3] for i in s[::-1]]  # query에 해당하는 문서들만 추출하여 정렬

  ##
  
  # for j in range(100):
  #   corpus[q][j][3]= sorted_has_answer
  #   print(corpus[q][j][3])
  # print("------")
  # corpus[q] = sorted(corpus[q], key=lambda x: doc_scores[s[::-1]].tolist().index(x['text']), reverse=True)
  # corpus[q] = sorted(corpus[q], key=lambda x: x in doc_scores[s][::-1], reverse=True)
  print("ITERATION: ", q)
  #print(sorted_has_answer)
  # sorted_corpus = [corpus[q][i] for i in s[::-1]]
  # print(sorted_corpus[q][0])
  # ctxs = [{
  #       'id': sorted_corpus[q][i][0] for i in s[::-1],
  #       'title': sorted_corpus[q][i][1] for i in s[::-1],
  #       'text': sorted_corpus[q][i][2] for i in s[::-1],
  #       'bm25_score': doc_scores[s][::-1],
  #       'has_answer': sorted_corpus[q][i][3] for i in s[::-1] }]

  # result.append({
  #       "question": query,
  #       "answers": answer,
  #       "ctxs": ctxs
  # })
  for k in range(100):
    data[q]["ctxs"][k]["id"]=sorted_id[k]
    data[q]["ctxs"][k]["title"]=sorted_title[k]
    data[q]["ctxs"][k]["text"]=sorted_text[k]
    data[q]["ctxs"][k]["score"]=sorted_scores[k]
    data[q]["ctxs"][k]["has_answer"]=sorted_has_answer[k]
# json_data = json.dumps(result)

# with open('NQ_test_benchmark_final_bm25.json', 'w') as file:
#     file.write(json_data)
#   # 결과를 리스트에 추가
#   for doc in sorted_corpus:
#     print(doc)
  # print("######################################")
  
  
  # data['key'] = 'new value'
  # print(corpus[q])
  #   print("\n")

  # print(corpus[q])
  # # bm25_scores.append(doc_scores) #append 된거 확인 
  # # print(bm25.get_top_n(tokenized_query))
  # # print(bm25_scores) #100개 [0]에 100개 다 들어감 
  # # count = 0
  # with open('NQ_test_benchmark_final_bm25.json', 'w') as file:
  #    json.dump(data, file)


k = 10

print(f'Accuracy@{k}: {calculate_accuracy(data, k):.4f}')
print(f'Precision@{k}: {calculate_precision(data, k):.4f}')
print(f'MRR@{k}: {calculate_mrr(data, k):.4f}')
print(f'MAP@{k}: {calculate_map(data, k):.4f}')
print(f'nDCG@{k}: {calculate_ndcg(data, k):.4f}')

## json 파일명 변경 및 data range 설정 필요
with open('NQ_test_benchmark_reranked_bm25_qe3.json', 'w') as file:
      json.dump(data, file)

ITERATION:  0
ITERATION:  1
ITERATION:  2
ITERATION:  3
ITERATION:  4
ITERATION:  5
ITERATION:  6
ITERATION:  7
ITERATION:  8
ITERATION:  9
ITERATION:  10
ITERATION:  11
ITERATION:  12
ITERATION:  13
ITERATION:  14
ITERATION:  15
ITERATION:  16
ITERATION:  17
ITERATION:  18
ITERATION:  19
ITERATION:  20
ITERATION:  21
ITERATION:  22
ITERATION:  23
ITERATION:  24
ITERATION:  25
ITERATION:  26
ITERATION:  27
ITERATION:  28
ITERATION:  29
ITERATION:  30
ITERATION:  31
ITERATION:  32
ITERATION:  33
ITERATION:  34
ITERATION:  35
ITERATION:  36
ITERATION:  37
ITERATION:  38
ITERATION:  39
ITERATION:  40
ITERATION:  41
ITERATION:  42
ITERATION:  43
ITERATION:  44
ITERATION:  45
ITERATION:  46
ITERATION:  47
ITERATION:  48
ITERATION:  49
ITERATION:  50
ITERATION:  51
ITERATION:  52
ITERATION:  53
ITERATION:  54
ITERATION:  55
ITERATION:  56
ITERATION:  57
ITERATION:  58
ITERATION:  59
ITERATION:  60
ITERATION:  61
ITERATION:  62
ITERATION:  63
ITERATION:  64
ITERATION:  65
ITERATION:  66
ITERA

In [18]:
k = 5

print(f'Accuracy@{k}: {calculate_accuracy(data, k):.4f}')
print(f'Precision@{k}: {calculate_precision(data, k):.4f}')
#print(f'Precision2@{k}: {calculate_precision2(data, k):.4f}')
print(f'MRR@{k}: {calculate_mrr(data, k):.4f}')
print(f'MAP@{k}: {calculate_map(data, k):.4f}')
print(f'nDCG@{k}: {calculate_ndcg(data, k):.4f}')

Accuracy@5: 0.4301
Precision@5: 0.1417
MRR@5: 0.2795
MAP@5: 0.2698
nDCG@5: 0.1880


In [19]:
k = 1

print(f'Accuracy@{k}: {calculate_accuracy(data, k):.4f}')
print(f'Precision@{k}: {calculate_precision(data, k):.4f}')
#print(f'Precision2@{k}: {calculate_precision2(data, k):.4f}')
print(f'MRR@{k}: {calculate_mrr(data, k):.4f}')
print(f'MAP@{k}: {calculate_map(data, k):.4f}')
print(f'nDCG@{k}: {calculate_ndcg(data, k):.4f}')

Accuracy@1: 0.1985
Precision@1: 0.0348
MRR@1: 0.1985
MAP@1: 0.1985
nDCG@1: 0.0993


In [ ]:
import pandas as pd
import math
import numpy as np
from collections import defaultdict

class BM25:
    def __init__(self, documents, k1=1.2, b=0.75):
        self.documents = documents
        self.k1 = k1
        self.b = b
        self.doc_count = len(documents) # 총 문서 갯수
        self.avgdl = 0                  # 문서 평균 길이 
        self.doc_freqs = []             # 각 문서 내 corpus 카운트 
        self.idf = {}                   # corpus당 idf 값
        self.doc_len = []               # 각 문서 길이
        
        self._calc_idf()

        
    def _calc_idf(self):
        doc_tokenized_corpus = list(map(lambda s: ' '.join(mecab.morphs(s)), self.documents ))
        doc_corpus = [token.split(" ") for token in doc_tokenized_corpus]

        # 문서별 IDF 계산
        word_freq = defaultdict(int)
        total_words = 0
        for document in doc_corpus:
            self.doc_len.append(len(document))
            total_words += len(document)
            
            frequencies = defaultdict(int)
            for word in document:
                frequencies[word] += 1
            
            self.doc_freqs.append(frequencies)
            
            for word, freq in frequencies.items():
                word_freq[word] += 1

        self.avgdl = total_words / self.doc_count
        
        for word, freq in word_freq.items():
            idf = math.log(1 + (self.doc_count - freq + 0.5) / (freq + 0.5))
            self.idf[word] = idf

            
    def get_top_n(self, query, n=5):
        query_morphs = mecab.morphs(query)
        scores = self.get_scores(query_morphs)
        top_n = np.argsort(scores)[::-1][:n]
        return [self.documents[i] for i in top_n]



    def get_scores(self, query):
        score = np.zeros(self.doc_count)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(corpus[q] or 0) for corpus in self.doc_freqs])
            # TF 계산
            try:
                score += (self.idf[q] or 0) * (q_freq * (self.k1 + 1) /
                                                   (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
            except Exception:
                # 알수 없는 단어일 경우 패스 
                pass
        return score